In [1]:
'''Main'''
import numpy as np
import pandas as pd

'''Data Prep'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

# Timer
import time as time

In [2]:
# read data
data = pd.read_csv('credit_card.csv')
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [3]:
# Create the feature matrix X and the labels array Y
dataX = data.copy().drop(['Class'],axis=1)
dataY = data['Class'].copy()

In [4]:
# We will create three data frames and check the effects of scaling in XGBoost
# Standardize X
from sklearn import preprocessing as pp
from sklearn.preprocessing import MinMaxScaler

# Case 1:
# Step 1: you drop the features that you do not want to scale
featuresToScale = dataX.drop(['Time'],axis=1).columns
# Step 2: create the scaler object
sX = pp.StandardScaler(copy=True)
# Step 3: create a new data frame to save the results
dataX1=dataX.copy()
# Step 4: transform the original data and save to dataX1
dataX1.loc[:,featuresToScale] = sX.fit_transform(dataX[featuresToScale])
dataX1.describe()
# now all rows have mean=0 and stderr=1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05
mean,94813.859575,-8.157366e-16,3.154853e-17,-4.409878e-15,-6.734811e-16,-2.874435e-16,4.168992e-16,-8.767997e-16,-2.423604e-16,3.078727e-16,...,2.754870e-16,1.685077e-17,1.478472e-15,-6.797197e-16,1.234659e-16,-7.659279e-16,3.247603e-16,-2.953495e-18,5.401572e-17,3.202236e-16
std,47488.145955,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,0.000000,-2.879855e+01,-4.403529e+01,-3.187173e+01,-4.013919e+00,-8.240810e+01,-1.963606e+01,-3.520940e+01,-6.130252e+01,-1.222802e+01,...,-7.069146e+01,-4.741907e+01,-1.506565e+01,-7.175446e+01,-4.683638e+00,-1.975033e+01,-5.401098e+00,-5.590660e+01,-4.674612e+01,-3.532294e-01
25%,54201.500000,-4.698918e-01,-3.624707e-01,-5.872142e-01,-5.993788e-01,-5.010686e-01,-5.766822e-01,-4.478860e-01,-1.746805e-01,-5.853631e-01,...,-2.746334e-01,-3.109433e-01,-7.473476e-01,-2.591784e-01,-5.854676e-01,-6.084001e-01,-6.780717e-01,-1.755053e-01,-1.604440e-01,-3.308401e-01
50%,84692.000000,9.245351e-03,3.965683e-02,1.186124e-01,-1.401724e-02,-3.936682e-02,-2.058046e-01,3.241723e-02,1.871982e-02,-4.681169e-02,...,-8.104705e-02,-4.009429e-02,9.345377e-03,-1.792420e-02,6.765678e-02,3.183240e-02,-1.081217e-01,3.325174e-03,3.406368e-02,-2.652715e-01
75%,139320.500000,6.716939e-01,4.867202e-01,6.774569e-01,5.250082e-01,4.433465e-01,2.991625e-01,4.611107e-01,2.740785e-01,5.435305e-01,...,1.725733e-01,2.537392e-01,7.283360e-01,2.364319e-01,7.257153e-01,6.728006e-01,4.996663e-01,2.255648e-01,2.371526e-01,-4.471707e-02
max,172792.000000,1.253351e+00,1.335775e+01,6.187993e+00,1.191874e+01,2.521413e+01,5.502015e+01,9.747824e+01,1.675153e+01,1.419494e+01,...,5.113464e+01,3.703471e+01,1.447304e+01,3.607668e+01,7.569684e+00,1.442532e+01,7.293975e+00,7.831940e+01,1.025434e+02,1.023622e+02


In [5]:
# Case 2: Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler

# Case 2:
# Step 1: you drop the features that you do not want to scale
featuresToScale = dataX.drop(['Time'],axis=1).columns
# Step 2: create the scaler object
mX = pp.MinMaxScaler(copy=True)
# Step 3: create a new data frame to save the results
dataX2=dataX.copy()
# Step 4: transform the original data and save to dataX1
dataX2.loc[:,featuresToScale] = mX.fit_transform(dataX[featuresToScale])
dataX2.describe()
# now all rows have min=0 and max=1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,...,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000,284807.000000
mean,94813.859575,0.958294,0.767258,0.837414,0.251930,0.765716,0.263020,0.265356,0.785385,0.462780,...,0.580265,0.561480,0.510031,0.665434,0.382234,0.577907,0.425448,0.416511,0.313124,0.003439
std,47488.145955,0.033276,0.017424,0.026275,0.062764,0.009292,0.013395,0.007537,0.012812,0.037846,...,0.008208,0.011841,0.033854,0.009274,0.081611,0.029261,0.078771,0.007450,0.006698,0.009736
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,54201.500000,0.942658,0.760943,0.821985,0.214311,0.761060,0.255295,0.261980,0.783148,0.440626,...,0.578011,0.557798,0.484730,0.663030,0.334454,0.560104,0.372036,0.415203,0.312049,0.000218
50%,84692.000000,0.958601,0.767949,0.840530,0.251050,0.765351,0.260263,0.265600,0.785625,0.461008,...,0.579600,0.561005,0.510347,0.665267,0.387756,0.578838,0.416932,0.416536,0.313352,0.000856
75%,139320.500000,0.980645,0.775739,0.855213,0.284882,0.769836,0.267027,0.268831,0.788897,0.483350,...,0.581682,0.564484,0.534688,0.667626,0.441460,0.597593,0.464807,0.418191,0.314712,0.003004
max,172792.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Now we use a nicer function to do classification
# This function includes K-fold cross validation
# This function output both predicted class and predicted prob.
# This function also generates all common performance metrics of classification
def build_model(data, features, target, classifier, oversample=None):
    '''
    Function that takes in data with features and target columns,
    builds a model using the specified classifier,
    and outputs the classification performance.
    
    Inputs:
        data (pd.DataFrame): Dataframe containing features and target columns for model building.
        features (list): List of features to use in model building.
        target (str): Name of target / label column.
        classifier (sklearn classifier Object): Model to build.
        
    Outputs:
        cm (pd.DataFrame): Confusion matrix.
        results (dict): Accuracy, Precision, Recall, F1 Score, AUC.
    '''
    
    # initialise stratified K Fold, 5 fold in this case
    kf = StratifiedKFold(n_splits=5, shuffle=False)
    # get splitting iterations
    kf.get_n_splits(X=data[features], y=data[target])
    
    # lists to store results
    y_pred_all = []
    y_true_all = []
    y_proba_all = []
    
    # comprehensive dictionary of results
    results = dict()
    
    # generate indices to split data
    for train_index, test_index in kf.split(data[features], data[target]):
        ### train & test for current iteration ###
        curr_train, curr_test = data.iloc[train_index], data.iloc[test_index]
        # train set
        X_train = curr_train[features]
        y_train = curr_train[target]
        # test set
        X_test = curr_test[features]
        y_test = curr_test[target]
        
        ### handling class imbalance ###
        if oversample:
            X_train, y_train = oversample.fit_resample(X_train, y_train)
        
        # classifier
        clf = classifier
        # fit data
        clf.fit(X_train, y_train)
        # get predictions
        y_pred = clf.predict(X_test) # get predicted class
        y_proba = clf.predict_proba(X_test) # get predicted prob.
        # extend list
        y_pred_all.extend(y_pred)
        y_true_all.extend(y_test)
        y_proba_all.extend(y_proba[:, 1])
        
    # generate confusion matrix
    cm = pd.DataFrame(confusion_matrix(y_true=y_true_all, y_pred=y_pred_all), \
                      index=['Actual: 0', 'Actual: 1'], \
                      columns=['Pred: 0', 'Pred: 1'])

    # generate results
    report = classification_report(y_true=y_true_all, y_pred=y_pred_all, output_dict=True)
    results['accuracy'] = report['accuracy']
    results['recall'] = report['1']['recall']
    results['precision'] = report['1']['precision']
    results['f1'] = report['1']['f1-score']
    
    # auc
    fpr, tpr, thresholds = roc_curve(y_true=y_true_all, y_score=y_proba_all)
    auc_score = auc(fpr, tpr)
    results['auc'] = auc_score
    
    return cm, results

In [7]:
# an simple example how to use this self-defined function
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()

# initialise decision tree classifier
dt = DecisionTreeClassifier(random_state=1)

# define train data frame, list of features, and target column name
features = list(dataX.columns)
target = 'Class'
train = pd.concat([dataX, dataY], axis=1)

# build model
cm_dt, results_dt = build_model(train, features, target, dt)

print("Confusion Matrix: \n\n", cm_dt)
print("\n")
print(f"F1 Score: {results_dt['f1']}")
print("--- %s seconds ---" % (time.time() - start_time))

Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   227529    56786
Actual: 1      136      356


F1 Score: 0.01235381892632821
--- 73.89886808395386 seconds ---


In [8]:
start_time = time.time()
train1 = pd.concat([dataX1, dataY], axis=1) # only this is different
# we use standardized input features, not raw data

# build model
cm_dt1, results_dt1 = build_model(train1, features, target, dt)

print("Confusion Matrix: \n\n", cm_dt1)
print("\n")
print(f"F1 Score: {results_dt1['f1']}")
print("--- %s seconds ---" % (time.time() - start_time))

Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   227529    56786
Actual: 1      136      356


F1 Score: 0.01235381892632821
--- 73.73727607727051 seconds ---


In [9]:
start_time = time.time()
train2 = pd.concat([dataX2, dataY], axis=1) 
# we use min-max scaled input features, not raw data

# build model
cm_dt2, results_dt2 = build_model(train2, features, target, dt)

print("Confusion Matrix: \n\n", cm_dt2)
print("\n")
print(f"F1 Score: {results_dt2['f1']}")
print("--- %s seconds ---" % (time.time() - start_time))

Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   227391    56924
Actual: 1      135      357


F1 Score: 0.012358714278296089
--- 70.00598645210266 seconds ---


In [10]:
# We can easily switch to other classification algorithms now
# XGBoost
from xgboost import XGBClassifier
start_time = time.time()
# initialise XGBoost classifier, mostly default values
xgb = XGBClassifier(random_state=1, use_label_encoder=False, eval_metric='error')

# build model
cm_xgb, results_xgb = build_model(train, features, target, xgb)
cm_xgb1, results_xgb1 = build_model(train1, features, target, xgb)
cm_xgb2, results_xgb2 = build_model(train2, features, target, xgb)

print("Confusion Matrix: \n\n", cm_xgb)
print("\n")
print(f"F1 Score: {results_xgb['f1']}")
print("Confusion Matrix: \n\n", cm_xgb1)
print("\n")
print(f"F1 Score: {results_xgb1['f1']}")
print("Confusion Matrix: \n\n", cm_xgb2)
print("\n")
print(f"F1 Score: {results_xgb2['f1']}")
print("--- %s seconds ---" % (time.time() - start_time))
# This is clearly better than DT

Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   282678     1637
Actual: 1      106      386


F1 Score: 0.30695825049701786
Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   282678     1637
Actual: 1      106      386


F1 Score: 0.30695825049701786
Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   282549     1766
Actual: 1      109      383


F1 Score: 0.2900416508898145
--- 363.9331612586975 seconds ---


In [11]:
# We switch to LightGBM and combine 3 cases of codes here
start_time = time.time()
import lightgbm as lgb

# initialize LightGBM, all default values
clf = lgb.LGBMClassifier()

# build model
cm_lgb, results_lgb = build_model(train, features, target, clf)
cm_lgb1, results_lgb1 = build_model(train1, features, target, clf)
cm_lgb2, results_lgb2 = build_model(train2, features, target, clf)

print("Confusion Matrix: \n\n", cm_lgb)
print("\n")
print(f"F1 Score: {results_lgb['f1']}")
print("Confusion Matrix: \n\n", cm_lgb1)
print("\n")
print(f"F1 Score: {results_lgb1['f1']}")
print("Confusion Matrix: \n\n", cm_lgb2)
print("\n")
print(f"F1 Score: {results_lgb2['f1']}")
print("--- %s seconds ---" % (time.time() - start_time))

Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   283026     1289
Actual: 1      269      223


F1 Score: 0.22255489021956093
Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   280568     3747
Actual: 1      181      311


F1 Score: 0.1367032967032967
Confusion Matrix: 

            Pred: 0  Pred: 1
Actual: 0   280376     3939
Actual: 1      140      352


F1 Score: 0.14718795734894416
--- 26.561797380447388 seconds ---
